In [148]:
import datetime

from google.cloud import speech
from difflib import SequenceMatcher

In [149]:
audio_file_path = "input_examples/test.wav"
script_text = """
Опрошенные называли и некоторые распространенные страхи, которые возникают в преклонном возрасте (а в случае старческой деменции могут превращаться в навязчивый бред): это боязнь нищеты, краж, потери памяти и другие тревожные ощущения. А главным проявлением старости почти все назвали потерю независимости — как в физическом, так и в ментальном смысле.
""".strip()

In [150]:
client = speech.SpeechClient()
with open(audio_file_path, "rb") as audio_file:
    content = audio_file.read()

audio = speech.RecognitionAudio(content=content)

if audio_file_path.endswith('.wav'):
    encoding = speech.RecognitionConfig.AudioEncoding.LINEAR16
elif audio_file_path.endswith('.flac'):
    encoding = speech.RecognitionConfig.AudioEncoding.FLAC
else:
    raise RuntimeError('Не поддерживаемый тип файла. Используйте WAV/FLAC')

# noinspection PyTypeChecker
config = speech.RecognitionConfig(
    encoding=encoding,
    audio_channel_count=2,
    language_code="ru-RU",
    enable_word_time_offsets=True,
    enable_automatic_punctuation=True,
)

In [151]:
response = client.recognize(config=config, audio=audio)

AttributeError: 'SpeechClient' object has no attribute 'st'

In [158]:
type(response.results[0])

google.cloud.speech_v1.types.cloud_speech.SpeechRecognitionResult

In [152]:
transcribed_text = response.results[0].alternatives[0].transcript

In [153]:
words_timestamps = response.results[0].alternatives[0].words

In [154]:
transcribed_text_words = transcribed_text.split(' ')
script_text_words = script_text.split(' ')

matcher = SequenceMatcher(None, transcribed_text_words, script_text_words)
errors: list[tuple[str, datetime.timedelta, str]] = []


for tag, start1, end1, start2, end2 in matcher.get_opcodes():
    if tag == 'insert':  # строка в сценарии, отсутствующая в транскрибированном тексте
        timecode = words_timestamps[start1].start_time
        errors.append(('потеряшка', timecode, ' '.join(script_text_words[start2:end2])))

In [155]:
errors

[('потеряшка',
  datetime.timedelta(seconds=6, microseconds=600000),
  '(а в случае старческой деменции могут превращаться в навязчивый бред): это боязнь нищеты, краж, потери памяти и другие тревожные ощущения. А')]

In [156]:
d = matcher.get_opcodes()
d

[('equal', 0, 4, 0, 4),
 ('replace', 4, 5, 4, 5),
 ('equal', 5, 11, 5, 11),
 ('insert', 11, 11, 11, 32),
 ('equal', 11, 13, 32, 34),
 ('delete', 13, 14, 34, 34),
 ('equal', 14, 17, 34, 37),
 ('replace', 17, 18, 37, 38),
 ('equal', 18, 19, 38, 39),
 ('replace', 19, 21, 39, 44),
 ('equal', 21, 22, 44, 45),
 ('replace', 22, 23, 45, 47),
 ('equal', 23, 24, 47, 48),
 ('replace', 24, 25, 48, 49)]